In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !zip -r /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000.zip /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/

In [3]:
# !ls /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000.zip

# Import Statements 

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure 
import json
import datetime
import copy
from PIL import Image as im 
import joblib


In [5]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [6]:
# Saving and Loading models using joblib 
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)

# paths 

In [8]:
val_prrocessed_img_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/processed_images/val/"
test_prrocessed_img_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/processed_images/train/"
train_prrocessed_img_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/processed_images/test/"

original_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam"

# One hot encoded train,val
one_hot_ans_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/encoded_answers.pkl"
label_encoder_object_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/labelencoder.pkl"

# Tokenizer object path 
tokenizer_obj_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/text_tokenizer.pkl"

# Preprocessed questions path train,val
que_emb_path = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/tokenised_data_post.pkl"

# vocab embedding
vocab_emb_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/vocab_embedding.pkl"

# path to load image and que mapping (yes/no)
train_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/data/train_yes_no.pkl"
val_pth = "/content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/data/val_yes_no.pkl"

# Load Objects , lables and pre processed questions

In [9]:
tokenizer_obj = load(tokenizer_obj_pth)
encoder_onj = load(label_encoder_object_pth)

In [14]:
# # Loading label encoding answers 
# # train_labels ,val_labels = np.load(one_hot_ans_pth)
# # train_lables.shape

# with open(one_hot_ans_pth, mode='rb') as f:
#     train_labels ,val_labels = pickle.load(f)
# train_labels = train_labels.astype('int64') 
# val_labels = val_labels.astype('int64') 
# train_labels.shape , val_labels.shape

In [12]:
# # Loading Processed Questions 
# with open(que_emb_path, mode='rb') as f:
#     train_que ,val_que = pickle.load(f)

# train_que.shape , val_que.shape

In [13]:
vocab_emb = load(vocab_emb_pth)
vocab_emb.shape

(8910, 100)

In [ ]:
train_df = pd.DataFrame(load(train_pth))
val_df = pd.DataFrame(load(val_pth))

train_df.head() 

,ques_id,img_id,question,ans,question_type,answers,answer_type
0,458752003,458752,Is this man a professional baseball player?,yes,is this,yes;yes;yes;yes;yes;no;yes;yes;yes;yes,yes/no
1,524291001,524291,Is the dog waiting?,yes,is the,yes;yes;yes;yes;yes;yes;yes;yes;yes;yes,yes/no
2,393223000,393223,Is the window open?,yes,is the,yes;no;yes;no;yes;yes;yes;yes;yes;yes,yes/no
3,393223003,393223,Is she brushing?,yes,is,yes;yes;yes;yes;yes;yes;yes;yes;yes;yes,yes/no
4,393224000,393224,Is the man smiling?,no,is the man,no;no;no;no;no;no;no;no;no;no,yes/no


In [ ]:
train_df.shape

(80789, 7)

# Data Loaders and transforms 

In [ ]:
# Load preprocessed question from here 
class Get_Image_Que(Dataset):
    """ dataset."""

    def __init__(self, que_arr, label_arr , data_file , root_dir, transform=None):
      # data file is a pandas dataframe containing all image ids and all question ids along with question and answers 
        
        self.data_file = data_file
        self.root_dir = root_dir
        self.transform = transform
        self.que_arr = que_arr
        self.label_arr = label_arr

        self.img_ids = data_file['img_id'].to_numpy()
        self.que_ids = data_file['ques_id'].to_numpy()

    def __len__(self):
        return len(self.data_file)

    def __getitem__(self, idx):
        # print(idx)
        # loading numpy preprocessed features of image 
        img_name = os.path.join(self.root_dir, str(self.img_ids[idx]) + ".jpg.npy")
        img_tensor = torch.tensor(np.load(img_name))

        # loading the question 
        # curr_que_id = self.que_ids[idx]
        curr_que = self.que_arr[idx]
        curr_ans = torch.tensor(self.label_arr[idx])
        
        if self.transform:
            # sample = self.transform(sample)
            pass

        return (img_tensor,curr_que,curr_ans)

In [ ]:
batch = 64
num = 128 
train_df.iloc[:num].value_counts()
train_data = Get_Image_Que(train_que[:num], train_labels[:num], train_df.iloc[:num],"/content/drive/MyDrive/SUB/PROJECTS/Dl/processed_images/train/")
train_loader = DataLoader(train_data, batch_size=batch, shuffle=True)

val_data = Get_Image_Que(val_que[:num], val_labels[:num], val_df.iloc[:num],"/content/drive/MyDrive/SUB/PROJECTS/Dl/processed_images/val/")
val_loader = DataLoader(val_data, batch_size=batch, shuffle=True)

In [ ]:
# next(iter(train_loader))

# Model Architecture 

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [ ]:
# #### 1000 answers data TYPE DATA ####
# from torch import nn, optim
# import torch.nn.functional as F

# class ImageModel(nn.Module):
#     def __init__(self):
#         super(ImageModel, self).__init__()
#         self.fc1 = nn.Linear(4096, 1024)
        
#     def forward(self, x):
#         x = self.fc1(x)
#         x = F.relu(x)
#         return x
    

# class TextModel(nn.Module):
#     def __init__(self, weights_matrix,hidden_size, num_layers):
#         super(TextModel, self).__init__()

#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
#         self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
#         self.embedding_dim = embedding_dim
#         # -> x needs to be: (batch_size, seq, input_size)
#         self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
#         self.fc1 = nn.Linear(1536, 1024)
        
        
#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
#         c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
#         x, _ = self.lstm(self.embedding(x), (h0,c0))  
#         # take hidden value from only last time stamp 
#         x = x[:, -1, :]
#         # make sure input tensor is flattened
#         x = x.view(x.shape[0], -1)
#         x =  F.relu(self.fc1(x))
#         return x

# class CombinedModel(nn.Module):
#     def __init__(self,  ImageModel, TextModel):
#         super(CombinedModel, self).__init__()
        
#         self.ImageModel = ImageModel
#         self.TextModel = TextModel

#         self.bn1 = nn.BatchNorm2d(num_features=1024)
#         self.drop = nn.Dropout(p=0.5)
#         self.fc1 = nn.Linear(1024,1000)
#         self.classifier = nn.Linear(1000, 2)
        
        
#     def forward(self, x1, x2):
#         x1 = self.ImageModel(x1)
#         x2 = self.TextModel(x2)
#         x2 = x2.view(x2.shape[0], -1)
#         # element wise multiplication 
#         x3 = torch.mul(x1,x2)
#         x3 = self.bn1(x3)
#         x3 = self.drop(x3)
        
#         x3 = F.relu(self.fc1(x3))\

#         output = F.log_softmax(self.classifier(x3), dim=1)
        
#         return output

# # # Create models and load state_dicts    
# # image_ml = ImageModel().to(device)
# # text_ml = TextModel(64,2).to(device)

# # # Load state dicts
# # # image_ml.load_state_dict(torch.load(PATH))
# # # text_ml.load_state_dict(torch.load(PATH))


# #### STORE THE MODEL ALONG WITH STATE CODE #####
# #### MOVE MODEL TO GPU CODE ####


# Loss Function and optimizers


In [ ]:
# Device configuration 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
#### 1000 answers data TYPE DATA ####

from torch import nn, optim
import torch.nn.functional as F
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.fc1 = nn.Linear(4096, 1024)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        return x
    

class TextModel(nn.Module):
    def __init__(self, weights_matrix,hidden_size, num_layers):
        super(TextModel, self).__init__()

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.embedding_dim = embedding_dim
        # -> x needs to be: (batch_size, seq, input_size)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(1536, 1024)
        
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        #print("embedding" ,self.embedding(x).shape)
        x, _ = self.lstm(self.embedding(x), (h0,c0))
        # take hidden value from only last time stamp 
        # x = x[:, -1, :]
        # make sure input tensor is flattened
        #x = torch.tensor(x)
        
        x = torch.reshape(x, (x.shape[0], -1))
        #print("after flatten" ,x.shape)
        x =  F.relu(self.fc1(x))
        return x

class CombinedModel(nn.Module):
    def __init__(self,  ImageModel, TextModel):
        super(CombinedModel, self).__init__()
        
        self.ImageModel = ImageModel
        self.TextModel = TextModel

        self.bn1 = nn.BatchNorm2d(num_features=1024)
        self.drop = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(2048,1024)
        self.fc2 = nn.Linear(1024,1000)
        self.classifier = nn.Linear(1000, 2)
        
        
    def forward(self, x1, x2):
        x1 = self.ImageModel(x1)
        #print("Image done")
        x2 = self.TextModel(x2)
        #print("text done")
        x2 = x2.view(x2.shape[0], -1)
        #print(x1.shape, x2.shape)
        # element wise multiplication 
        x3 = torch.cat((x1, x2), 1)
        # print(x3.shape )
        #print("Mul done")
        #x3 = self.bn1(x3)
        #x3 = self.drop(x3)
        
        x3 = F.relu(self.fc1(x3))
        x3 = F.relu(self.fc2(x3))
        output = self.classifier(x3)
        
        return output

# # Load state dicts
# # image_ml.load_state_dict(torch.load(PATH))
# # text_ml.load_state_dict(torch.load(PATH))


#### STORE THE MODEL ALONG WITH STATE CODE #####
#### MOVE MODEL TO GPU CODE ####
image_ml = ImageModel().to(device)
text_ml = TextModel(torch.tensor(vocab_emb),64, 2).to(device)

model = CombinedModel(image_ml, text_ml).to(device)

#criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# image_ml = ImageModel().to(device)
# text_ml = TextModel(torch.tensor(vocab_emb),64, 2).to(device)

# model = CombinedModel(image_ml, text_ml).to(device)
# criterion = nn.NLLLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop 

In [ ]:
epochs = 2
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for idx, (images, questions, labels) in enumerate(train_loader):
        # tokenize the question array 

        # images and labels and toknize output sent to device 
        images, questions, labels = images.to(device), questions.to(device), labels.to(device)
        #print(questions.shape)
        optimizer.zero_grad()
        
        log_ps = model(images, questions)
        loss = criterion(log_ps, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if idx % 100 == 0:
          print("Batch no: {}".format(idx))
          print(f'{idx/(len(train_que)/64)*100}% done..')
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for images, questions, labels in val_loader:
                images, questions, labels = images.to(device), questions.to(device), labels.to(device)
                
                log_ps = model(images, questions)
                test_loss = criterion(log_ps, torch.max(labels, 1)[1])
                
                ps = log_ps
                # print(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == torch.max(labels, 1)[1]
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        train_losses.append(running_loss/len(train_loader))
        test_losses.append(test_loss/len(val_loader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(val_loader)))
        
        checkpoint = {
        "epoch": e+1,
        "model_state": model.state_dict(),
        "optim_state": optimizer.state_dict(),
        "train_losses": train_losses,
        "test_losses": train_losses,
        "running_loss": running_loss
        }

        FILE = "checkpoint.pth" + str(e+1)
        torch.save(checkpoint, FILE)
       
        

Batch no: 0
0.0% done..
Epoch: 1/2..  Training Loss: 0.696..  Test Loss: 0.341..  Test Accuracy: 0.594
Batch no: 0
0.0% done..
Epoch: 2/2..  Training Loss: 0.705..  Test Loss: 0.341..  Test Accuracy: 0.547


In [ ]:
# model.train()

model = CombinedModel(image_ml, text_ml).to(device)
# criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
print(model.fc1.weight)
# loading a checkpoint 
# model = Model(n_input_features=6)
# optimizer = optimizer = torch.optim.SGD(model.parameters(), lr=0)

checkpoint = torch.load("checkpoint.pth1")
model.load_state_dict(checkpoint['model_state'])
optimizer.load_state_dict(checkpoint['optim_state'])
epoch = checkpoint['epoch']
print(model.fc1.weight)

# model.eval()

Parameter containing:
tensor([[ 0.0144,  0.0219,  0.0162,  ..., -0.0146,  0.0148, -0.0056],
        [ 0.0066,  0.0171, -0.0197,  ...,  0.0013, -0.0172, -0.0133],
        [ 0.0149,  0.0115,  0.0209,  ...,  0.0092, -0.0046, -0.0220],
        ...,
        [ 0.0193, -0.0177,  0.0162,  ...,  0.0172,  0.0022, -0.0183],
        [ 0.0067, -0.0051, -0.0024,  ..., -0.0080, -0.0167, -0.0079],
        [ 0.0049,  0.0029, -0.0040,  ..., -0.0050, -0.0137, -0.0023]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.0045, -0.0215, -0.0145,  ...,  0.0035,  0.0099,  0.0133],
        [-0.0045, -0.0009, -0.0056,  ...,  0.0100,  0.0079, -0.0190],
        [ 0.0267, -0.0021,  0.0190,  ..., -0.0027,  0.0155,  0.0039],
        ...,
        [-0.0034, -0.0057, -0.0169,  ..., -0.0135, -0.0027, -0.0214],
        [-0.0224,  0.0064, -0.0116,  ...,  0.0013, -0.0039, -0.0054],
        [ 0.0071,  0.0135,  0.0041,  ..., -0.0018,  0.0166, -0.0011]],
       device='cuda:0', requires_grad=True)


In [ ]:
!zip -r /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000.zip /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000

  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/ (stored 0%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/vqa_raw_val2014.json (deflated 74%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/vqa_raw_train2014.json (deflated 74%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/processed_questions.pkl (deflated 69%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/text_tokenizer.pkl (deflated 52%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/vocab_embedding.pkl (deflated 48%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/tokenised_data_post.pkl (deflated 90%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_1000/labelencoder.pkl (deflated 92%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/process

In [ ]:
!zip -r /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no.zip /content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no

  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/ (stored 0%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/vqa_raw_train2014.json (deflated 78%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/processed_questions.pkl (deflated 67%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/text_tokenizer.pkl (deflated 52%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/labelencoder.pkl (deflated 23%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/encoded_answers.pkl (deflated 96%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/tokenised_data_post.pkl (deflated 90%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no/vocab_embedding.pkl (deflated 48%)
  adding: content/drive/MyDrive/SUB/PROJECTS/Dl/Dataset

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0062,  0.0011, -0.0179,  ...,  0.0181,  0.0150,  0.0154],
        [-0.0097,  0.0082, -0.0201,  ...,  0.0023, -0.0103,  0.0005],
        [ 0.0168,  0.0050, -0.0114,  ...,  0.0140,  0.0025,  0.0073],
        ...,
        [-0.0167, -0.0168, -0.0068,  ..., -0.0161,  0.0204, -0.0020],
        [-0.0002, -0.0076,  0.0116,  ...,  0.0026,  0.0025, -0.0144],
        [-0.0170, -0.0053, -0.0207,  ..., -0.0117,  0.0188, -0.0072]],
       device='cuda:0', requires_grad=True)

In [ ]:
gdown https://drive.google.com/uc?id=1J0uE7D72GJQQKVx35ZTMDculmnbmte7Q
https://drive.google.com/file/d/1J0uE7D72GJQQKVx35ZTMDculmnbmte7Q/view?usp=sharing

In [ ]:
https://drive.google.com/file/d/1-6HzjlHh63mXdD1fshnlrm1WSoyeSc-c/view?usp=sharing
gdown https://drive.google.com/uc?id=1-6HzjlHh63mXdD1fshnlrm1WSoyeSc-c
content/drive/MyDrive//SUB/PROJECTS/Dl/Dataset/shivam/processed_data_yes_no

https://drive.google.com/drive/folders/1aGkSTbEuHXJwm100VvArKbpJfNZt5QMH?usp=sharing
https://drive.google.com/drive/folders/1aGkSTbEuHXJwm100VvArKbpJfNZt5QMH?usp=sharing

gdown https://drive.google.com/uc?id=1aGkSTbEuHXJwm100VvArKbpJfNZt5QMH?usp=sharing

In [ ]:
https://drive.google.com/drive/folders/1aGkSTbEuHXJwm100VvArKbpJfNZt5QMH?usp=sharing